<a href="https://colab.research.google.com/github/mudassir1231/API/blob/main/h2oGPT_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Question-Answer [h2oGPT](https://github.com/h2oai/h2ogpt)

In this notebook, we demonstrate how one can use h2oGPT with a large language model.

To begin, please get free ngrok account to get auth token (e.g.) using your Google email/login and get token: https://dashboard.ngrok.com/get-started/setup .  You will be asked for this token below in an input box.

In [1]:
!git clone https://github.com/h2oai/h2ogpt.git
!cd h2ogpt && git checkout 2668694581347b0d1afe76760213db46f7214126 -q
!cp -ar h2ogpt/. ./
!rm -r h2ogpt

Cloning into 'h2ogpt'...
remote: Enumerating objects: 13890, done.
remote: Counting objects: 100% (13890/13890), done.
remote: Compressing objects: 100% (4021/4021), done.
remote: Total 13890 (delta 9769), reused 10798 (delta 9713), pack-reused 0
Receiving objects: 100% (13890/13890), 23.72 MiB | 21.16 MiB/s, done.
Resolving deltas: 100% (9769/9769), done.


In [2]:
# Install pyhon 3.10 that will be used within pipenv
!sudo add-apt-repository ppa:deadsnakes/ppa -y > /dev/null
!sudo apt install python3.10 python3.10-distutils psmisc -y > /dev/null
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10 > /dev/null

In [3]:
# Install dependencies
!for fil in requirements.txt reqs_optional/requirements_optional_langchain.txt reqs_optional/requirements_optional_gpt4all.txt reqs_optional/requirements_optional_langchain.gpllike.txt reqs_optional/requirements_optional_langchain.urls.txt ; do pip install -r $fil ; done


  Cloning https://github.com/huggingface/peft.git (to revision 0b62b4378b4ce9367932c73540349da9a41bdea8) to /tmp/pip-req-build-5mfqh_k5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-5mfqh_k5
  Running command git rev-parse -q --verify 'sha^0b62b4378b4ce9367932c73540349da9a41bdea8'
  Running command git fetch -q https://github.com/huggingface/peft.git 0b62b4378b4ce9367932c73540349da9a41bdea8
  Running command git checkout -q 0b62b4378b4ce9367932c73540349da9a41bdea8
  Resolved https://github.com/huggingface/peft.git to commit 0b62b4378b4ce9367932c73540349da9a41bdea8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Ignoring pypandoc: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
  Obtaining dependency information for datasets==2.13.0 from https://files.pythonhosted.org/packages/17/d8/f808

In [4]:
# download llama model if running that:
!rm -rf WizardLM-7B-uncensored.ggmlv3.q8_0.bin*
!wget https://huggingface.co/TheBloke/WizardLM-7B-uncensored-GGML/resolve/main/WizardLM-7B-uncensored.ggmlv3.q8_0.bin

--2023-08-19 17:52:59--  https://huggingface.co/TheBloke/WizardLM-7B-uncensored-GGML/resolve/main/WizardLM-7B-uncensored.ggmlv3.q8_0.bin
Resolving huggingface.co (huggingface.co)... 65.8.178.118, 65.8.178.12, 65.8.178.27, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/c0/cd/c0cd768b4cd58780ae60ca18240a853723360aac1874854c9e07bc87d943ee47/2802e2c7ffb3cae9bab40425a2600d286b98ed5cc7fe2116fc205a2a101e913e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27WizardLM-7B-uncensored.ggmlv3.q8_0.bin%3B+filename%3D%22WizardLM-7B-uncensored.ggmlv3.q8_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1692726779&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MjcyNjc3OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9jMC9jZC9jMGNkNzY4YjRjZDU4NzgwYWU2MGNhMTgyNDBhODUzNzIzMzYwY

In [5]:
# Sign-up for free ngrok account using (e.g.) your Google email/login and get token: https://dashboard.ngrok.com/get-started/setup

!pip install pyngrok
import getpass
from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open an http ngrok tunnel
connection_string = ngrok.connect(7860, "http").public_url
print("Go to this address in about 20 seconds, and click on Visit Site: %s" % connection_string)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=98b1a2575cec9fd42d3e582aa6b0d0b3d134c4a45eb9d545864e06c93fcec267
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········


Go to this address in about 20 seconds, and click on Visit Site: https://e5a9-35-229-101-62.ngrok-free.app


In [6]:
get_ipython().system_raw("""GRADIO_SERVER_PORT=7860 python generate.py --base_model='llama' --prompt_type=wizard2 --score_model=None --langchain_mode=ChatLLM --visible_langchain_modes="['ChatLLM', 'UserData', 'MyData']" --user_path=user_path --share=False &> logs.txt &""")
# wait a bit for server to come up
import time
time.sleep(20)

In [7]:
# can see what is running:
# !ps -auxwf

In [8]:
# can check logs
# !cat logs.txt

In [9]:
# can kill old ngrok + generate and try again, or just restart entire runtime + run all cells
do_kill = False
if do_kill:
  !pkill -f generate --signal 9
  !pkill -f frpc_linux_amd --signal 9
  !pkill -f ngrok --signal 9